In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
fastq_ligation = snakemake.input['ligation']
ligation_chart = snakemake.output['ligation']
tags_counts = snakemake.input['tags']
tags_chart = snakemake.output['tags']

In [ ]:
def read_ligation(filename):
    data = pd.read_csv(filename)
    barcodes = pd.concat((data['sample'], data.filter(like='barcodes')), axis='columns')
    barcodes = barcodes.melt(id_vars='sample', var_name='Barcodes', value_name='Counts')
    barcodes['Barcodes'] = barcodes['Barcodes'].str.extract(r'(\d+)_barcodes').astype(int)
    totals = barcodes.groupby('sample').Counts.sum().rename('Percent')
    barcodes = barcodes.merge(totals, 'left', on='sample')
    barcodes['Percent'] = barcodes['Counts'] / barcodes['Percent'] * 100
    
    
    positions = pd.concat((data['sample'], data.filter(like='in_position')), axis='columns')
    positions = positions.melt(id_vars='sample', var_name='Position', value_name='Counts')
    positions['Position'] = positions['Position'].str.extract(r'in_position_(\d+)').astype(int)
    positions = positions.merge(totals, 'left', on='sample')
    positions['Percent'] = positions['Counts'] / positions['Percent'] * 100
    
    return barcodes, positions

def read_tag_counts(filename):
    data = pd.read_csv(filename)
    counts = np.zeros((data['position'].max(), 96))
    not_found = []
    name = []
    idx = np.arange(96)+1
    for i, (position, dat) in enumerate(data.groupby('position')):
        not_found.append(dat.loc[dat.tag == 'NOT_FOUND', 'counts'].iloc[0])
        dat = dat[dat.tag != 'NOT_FOUND']
        dat = pd.concat((dat, dat['tag'].str.extract(r'^(.*?)(?:_([A-H]))?(\d{1,2})$')), axis='columns').rename(columns={0: 'name', 1: 'row', 2: 'number'})
        dat['number'] = dat['number'].astype(int)
    
        names = dat.name.unique()
        if len(names) == 1:
            name.append(names[0])
        else:
            name.append("MULTIPLE")

        dat.sort_values(by=['row', 'number'], inplace=True)
        
        if not all(dat['row'].isna()):
            dat['number'] = dat['number'] + dat['row'].apply(lambda x: 12 * (ord(x) - ord('A')))
        counts[i][np.isin(idx, dat['number'])] = dat['counts']
    return not_found, name, counts.reshape((data['position'].max(), 8, 12))

In [ ]:
# plot ligation efficiencies
barcodes, positions = read_ligation(fastq_ligation)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.barplot(data=barcodes, x='Barcodes', y='Percent', ax=axes[0])
sns.barplot(data=positions, x='Position', y='Percent', ax=axes[1])
fig.savefig(ligation_chart)

In [ ]:
# plot the tag counts as a function of well position
not_founds, names, counts = read_tag_counts(tags_counts)
num_rows = int(np.ceil(len(not_founds) / 2))
fig, axes = plt.subplots(num_rows, 2, figsize=(14, 3 * num_rows + 2))
for not_found, name, cnts, ax in zip(not_founds, names, counts, axes.flatten()):
    sns.heatmap(cnts, xticklabels=list(range(1,13)), yticklabels='ABCDEFGH', ax=ax)
    ax.set_title(f'{name}, {not_found} missing')
fig.savefig(tags_chart)